### Task 1:
1. Train a U-2-Net model on this dataset after splitting it into training (15 images) and validation set (5 images).
2. You may choose any optimizer, loss function and augmentation you deem appropriate.
3. You are free to use any metrics, however, ‘dice_score` and IoU must be among them.
4. Keep model’s input_size as 320x320.


Work : <br>
Trained models with different loss function , optimizer , batchsize ....etc

Batch Size 100 and 40:<br>
BCE loss was better than IOu and Dice <br>

200 epoch val loss <br>
Bce  :  0.42 <br>
Dice & IOU: 0.55 approx

In [ ]:
!git clone https://github.com/xuebinqin/U-2-Net.git
    #load the repooo

Cloning into 'U-2-Net'...
remote: Enumerating objects: 1074, done.
remote: Counting objects: 100% (1074/1074), done.
remote: Compressing objects: 100% (548/548), done.
remote: Total 1074 (delta 538), reused 1036 (delta 522), pack-reused 0
Receiving objects: 100% (1074/1074), 66.94 MiB | 36.21 MiB/s, done.
Resolving deltas: 100% (538/538), done.


In [ ]:
%cd U-2-Net
%pwd

/kaggle/working/U-2-Net


'/kaggle/working/U-2-Net'

In [ ]:
import os
import torch
import torchvision
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.optim as optim
import torchvision.transforms as standard_transforms

import numpy as np
import glob
import os
import torch.nn.functional as F

from data_loader import Rescale
from data_loader import RescaleT
from data_loader import RandomCrop
from data_loader import ToTensor
from data_loader import ToTensorLab
from data_loader import SalObjDataset

from model import U2NET
from model import U2NETP
from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Chage to to your data path

In [ ]:
image_dir = "/kaggle/input/carmasks/dataset-20230907T065449Z-001/dataset/Image/"
label_dir = "/kaggle/input/carmasks/dataset-20230907T065449Z-001/dataset/Mask/"

In [ ]:
## config with all hyper parameter
class cfg:
  epoch_num = 20
  batch_size_train = 1
  batch_size_val = 1
  lr =0.001
  train_num = 0
  val_num = 0
  image_ext='.jpg'
  label_ext ='.png'
  model_name = 'u2netp'

cfg.image_dir = image_dir
cfg.label_dir = label_dir
cfg.model_dir  = os.getcwd()


In [ ]:
tra_img_name_list = glob.glob(cfg.image_dir + '*' + cfg.image_ext)
tra_lbl_name_list = []
for img_path in tra_img_name_list:
	img_name = img_path.split(os.sep)[-1]

	aaa = img_name.split(".")
	bbb = aaa[0:-1]
	imidx = bbb[0]
	for i in range(1,len(bbb)):
		imidx = imidx + "." + bbb[i]

	tra_lbl_name_list.append(cfg.label_dir + imidx + cfg.label_ext)

In [ ]:
len(tra_lbl_name_list)

20

Splting dataset 15:5

In [ ]:
tra_img_train_list, tra_img_val_list, tra_lbl_train_list, tra_lbl_val_list = train_test_split(
    tra_img_name_list, tra_lbl_name_list, test_size=5, random_state=42)

In [ ]:
print(len(tra_img_train_list))
print(tra_img_train_list)
print(tra_lbl_train_list)

15
['/kaggle/input/carmasks/dataset-20230907T065449Z-001/dataset/Image/9.jpg', '/kaggle/input/carmasks/dataset-20230907T065449Z-001/dataset/Image/12.jpg', '/kaggle/input/carmasks/dataset-20230907T065449Z-001/dataset/Image/35.jpg', '/kaggle/input/carmasks/dataset-20230907T065449Z-001/dataset/Image/49.jpg', '/kaggle/input/carmasks/dataset-20230907T065449Z-001/dataset/Image/14.jpg', '/kaggle/input/carmasks/dataset-20230907T065449Z-001/dataset/Image/34.jpg', '/kaggle/input/carmasks/dataset-20230907T065449Z-001/dataset/Image/10.jpg', '/kaggle/input/carmasks/dataset-20230907T065449Z-001/dataset/Image/17.jpg', '/kaggle/input/carmasks/dataset-20230907T065449Z-001/dataset/Image/2.jpg', '/kaggle/input/carmasks/dataset-20230907T065449Z-001/dataset/Image/41.jpg', '/kaggle/input/carmasks/dataset-20230907T065449Z-001/dataset/Image/11.jpg', '/kaggle/input/carmasks/dataset-20230907T065449Z-001/dataset/Image/46.jpg', '/kaggle/input/carmasks/dataset-20230907T065449Z-001/dataset/Image/39.jpg', '/kaggle/i

In [ ]:
#This is dataset function present in U2Net Util to convet it to correct format
salobj_train_dataset = SalObjDataset(
    img_name_list=tra_img_train_list,
    lbl_name_list=tra_lbl_train_list,
    transform=transforms.Compose([
        RescaleT(320),
        RandomCrop(288),
        ToTensorLab(flag=0)]))

salobj_val_dataset = SalObjDataset(
    img_name_list=tra_img_val_list,
    lbl_name_list=tra_lbl_val_list,
    transform=transforms.Compose([

        RescaleT(320),
        RandomCrop(288),
        ToTensorLab(flag=0)]))

In [ ]:
salobj_train_dataloader = DataLoader(salobj_train_dataset, batch_size=cfg.batch_size_train, shuffle=True, num_workers=1)
salobj_val_dataloader = DataLoader(salobj_val_dataset, batch_size=cfg.batch_size_val, shuffle=False, num_workers=1)

### Loss Functions::
Binary Cross-Entropy (BCE) Loss: BCE measures the dissimilarity between predicted and actual binary outcomes, commonly used for binary classification tasks.

Dice Loss: Dice loss quantifies the similarity between predicted and actual segmentation masks, often employed in medical image segmentation.

Intersection over Union (IoU) Loss: IoU loss evaluates the overlap between predicted and ground truth regions, frequently used in object detection and image segmentation to improve localization accuracy.

**Bce performed slightly better in low epoch**

In [ ]:
class DiceLoss(torch.nn.Module):
    def __init__(self, smooth=1.0):
        super(DiceLoss, self).__init__()
        self.smooth = smooth

    def forward(self, prediction, target):
        intersection = torch.sum(prediction * target)
        union = torch.sum(prediction) + torch.sum(target)
        dice_coeff = (2.0 * intersection + self.smooth) / (union + self.smooth)
        loss = 1.0 - dice_coeff
        return loss

In [ ]:
bce_loss = nn.BCELoss(size_average=True)

def muti_bce_loss_fusion(d0, d1, d2, d3, d4, d5, d6, labels_v):

	loss0 = bce_loss(d0,labels_v)
	loss1 = bce_loss(d1,labels_v)
	loss2 = bce_loss(d2,labels_v)
	loss3 = bce_loss(d3,labels_v)
	loss4 = bce_loss(d4,labels_v)
	loss5 = bce_loss(d5,labels_v)
	loss6 = bce_loss(d6,labels_v)

	loss = loss0 + loss1 + loss2 + loss3 + loss4 + loss5 + loss6
	print("l0: %3f, l1: %3f, l2: %3f, l3: %3f, l4: %3f, l5: %3f, l6: %3f\n"%(loss0.data.item(),loss1.data.item(),loss2.data.item(),loss3.data.item(),loss4.data.item(),loss5.data.item(),loss6.data.item()))

	return loss0, loss

/opt/conda/lib/python3.10/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [ ]:
def muti_Dice_loss_fusion(d0, d1, d2, d3, d4, d5, d6, labels_v):

	loss0 = DiceLoss(smooth=1e-5)(d0,labels_v)
	loss1 = DiceLoss(smooth=1e-5)(d1,labels_v)
	loss2 = DiceLoss(smooth=1e-5)(d2,labels_v)
	loss3 = DiceLoss(smooth=1e-5)(d3,labels_v)
	loss4 = DiceLoss(smooth=1e-5)(d4,labels_v)
	loss5 = DiceLoss(smooth=1e-5)(d5,labels_v)
	loss6 = DiceLoss(smooth=1e-5)(d6,labels_v)

	loss = loss0 + loss1 + loss2 + loss3 + loss4 + loss5 + loss6
	print("l0: %3f, l1: %3f, l2: %3f, l3: %3f, l4: %3f, l5: %3f, l6: %3f\n"%(loss0.data.item(),loss1.data.item(),loss2.data.item(),loss3.data.item(),loss4.data.item(),loss5.data.item(),loss6.data.item()))

	return loss0, loss

In [ ]:
#IOU loss for the item .....
class IoULoss(torch.nn.Module):
    def __init__(self, smooth=1.0):
        super(IoULoss, self).__init__()
        self.smooth = smooth

    def forward(self, prediction, target):
        intersection = torch.sum(prediction * target)
        union = torch.sum(prediction) + torch.sum(target) - intersection
        iou = (intersection + self.smooth) / (union + self.smooth)
        loss = 1.0 - iou
        return loss

def multi_IoU_loss_fusion(d0, d1, d2, d3, d4, d5, d6, labels_v):
    loss0 = IoULoss(smooth=1e-5)(d0, labels_v)
    loss1 = IoULoss(smooth=1e-5)(d1, labels_v)
    loss2 = IoULoss(smooth=1e-5)(d2, labels_v)
    loss3 = IoULoss(smooth=1e-5)(d3, labels_v)
    loss4 = IoULoss(smooth=1e-5)(d4, labels_v)
    loss5 = IoULoss(smooth=1e-5)(d5, labels_v)
    loss6 = IoULoss(smooth=1e-5)(d6, labels_v)

    loss = loss0 + loss1 + loss2 + loss3 + loss4 + loss5 + loss6
    print("l0: %3f, l1: %3f, l2: %3f, l3: %3f, l4: %3f, l5: %3f, l6: %3f\n" % (
        loss0.data.item(), loss1.data.item(), loss2.data.item(), loss3.data.item(), loss4.data.item(), loss5.data.item(),
        loss6.data.item()))

    return loss0, loss

In [ ]:
if(cfg.model_name=='u2net'):
    net = U2NET(3, 1)
elif(cfg.model_name=='u2netp'):
    net = U2NETP(3,1)
if torch.cuda.is_available():
    net.cuda()
# net.load_state_dict(torch.load('/kaggle/input/carmasks/u2netp.pth'))

#### Optimizer used :
Adam , Adamw , SGD

In [ ]:
print("---define optimizer...")
optimizer = optim.AdamW(net.parameters(), lr=cfg.lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)
# optimizer = optim.AdamW(net.parameters(), lr=cfg.lr, weight_decay=1e-4)
# optimizer = optim.Adam(net.parameters(), lr=cfg.lr, weight_decay=1e-4 ,eps=1e-08)
# optimizer = optim.SGD(net.parameters(),lr=cfg.lr,momentum=0.9. weight_decay=1e-4)

---define optimizer...


In [ ]:
ite_num = 0
running_loss = 0.0
running_tar_loss = 0.0
ite_num4val = 0
save_frq = 10
best_val_loss=999999 #largeno
print("---start training...")
for epoch in range(0, cfg.epoch_num):
    net.train()

    for i, data in enumerate(salobj_train_dataloader):
        ite_num = ite_num + 1
        ite_num4val = ite_num4val + 1

        inputs, labels = data['image'], data['label']

        inputs = inputs.type(torch.FloatTensor)
        labels = labels.type(torch.FloatTensor)

        # wrap them in Variable
        if torch.cuda.is_available():
            inputs_v, labels_v = Variable(inputs.cuda(), requires_grad=False), Variable(labels.cuda(),
                                                                                        requires_grad=False)
        else:
            inputs_v, labels_v = Variable(inputs, requires_grad=False), Variable(labels, requires_grad=False)

        # y zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        d0, d1, d2, d3, d4, d5, d6 = net(inputs_v)
        loss2, loss = multi_IoU_loss_fusion(d0, d1, d2, d3, d4, d5, d6, labels_v)

        loss.backward()
        optimizer.step()

        # # print statistics
        running_loss += loss.data.item()
        running_tar_loss += loss2.data.item()

        # del temporary outputs and loss
        del d0, d1, d2, d3, d4, d5, d6, loss2, loss

        print("[epoch: %3d/%3d, batch: %5d/%5d, ite: %d] train loss: %3f, tar: %3f " % (
        epoch + 1, cfg.epoch_num, (i + 1) * cfg.batch_size_train, cfg.train_num, ite_num, running_loss / ite_num4val, running_tar_loss / ite_num4val))

        if ite_num % save_frq == 0:

            torch.save(net.state_dict(), cfg.model_dir + cfg.model_name+"_bce_itr_%d_train_%3f_tar_%3fIOU20.pth" % (ite_num, running_loss / ite_num4val, running_tar_loss / ite_num4val))
            running_loss = 0.0
            running_tar_loss = 0.0
            net.train()  # resume train
            ite_num4val = 0

        ## validation
        # Validation loop
        net.eval()  # Set the model to evaluation mode
        val_loss = 0.0

        with torch.no_grad():
            for i, data in enumerate(salobj_val_dataloader):
                inputs, labels = data['image'], data['label']

                inputs = inputs.type(torch.FloatTensor)
                labels = labels.type(torch.FloatTensor)

                if torch.cuda.is_available():
                    inputs_v, labels_v = Variable(inputs.cuda(), requires_grad=False), Variable(labels.cuda(), requires_grad=False)
                else:
                    inputs_v, labels_v = Variable(inputs, requires_grad=False), Variable(labels, requires_grad=False)

                # Forward pass
                d0, d1, d2, d3, d4, d5, d6 = net(inputs_v)
                loss2, loss = multi_IoU_loss_fusion(d0, d1, d2, d3, d4, d5, d6, labels_v)

                val_loss += loss.data.item()

                # Clean up
                del d0, d1, d2, d3, d4, d5, d6, loss2, loss

        average_val_loss = val_loss / len(salobj_val_dataloader)
        print("[epoch: %3d/%3d] Validation Loss: %3f" % (epoch + 1, cfg.epoch_num, average_val_loss))

        # Save the model checkpoint if validation loss is lower than previous best
        if average_val_loss < best_val_loss:
            best_val_loss = average_val_loss
            torch.save(net.state_dict(), cfg.model_dir + cfg.model_name + "_best_IOULoss20.pth")

        net.train()  # S

---start training...


/opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:3737: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


l0: 0.746922, l1: 0.755349, l2: 0.739420, l3: 0.756333, l4: 0.711895, l5: 0.696587, l6: 0.784948

[epoch:   1/ 20, batch:     1/    0, ite: 1] train loss: 5.191454, tar: 0.746922 
l0: 0.746825, l1: 0.696495, l2: 0.706683, l3: 0.720831, l4: 0.721410, l5: 0.720846, l6: 0.719931

l0: 0.736935, l1: 0.717274, l2: 0.712053, l3: 0.723219, l4: 0.721479, l5: 0.720399, l6: 0.718785

l0: 0.839026, l1: 0.843660, l2: 0.837453, l3: 0.838739, l4: 0.837617, l5: 0.836796, l6: 0.837008

l0: 0.767535, l1: 0.757236, l2: 0.750673, l3: 0.758562, l4: 0.757124, l5: 0.756171, l6: 0.755304

l0: 0.765602, l1: 0.757694, l2: 0.752049, l3: 0.758360, l4: 0.756237, l5: 0.755102, l6: 0.754627

[epoch:   1/ 20] Validation Loss: 5.311148
l0: 0.773772, l1: 0.627296, l2: 0.629776, l3: 0.633143, l4: 0.593654, l5: 0.637912, l6: 0.725206

[epoch:   1/ 20, batch:     2/    0, ite: 2] train loss: 4.906107, tar: 0.760347 
l0: 0.756075, l1: 0.669868, l2: 0.704045, l3: 0.718609, l4: 0.720635, l5: 0.720390, l6: 0.716792

l0: 0.732

In [ ]:
# Model working Output Examin :

 # Model working Output Examin :

| Image | Output |
|-------|--------|
| ![Image](https://github.com/3112ik09/Images/blob/main/17.jpg?raw=true) | ![Output](https://github.com/3112ik09/Images/blob/main/U2NetTrain.png?raw=true) |

## Unseen Data
| Image | Output |
|-------|--------|
| ![Image](https://github.com/3112ik09/Images/blob/main/car1.jpg?raw=true) | ![Output](https://github.com/3112ik09/Images/blob/main/u2netCustom.png?raw=true) |

# Result:<br>
 **OVerfitted on the traning data** ->>
did not perform good in unseen data